<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

![](images/ucl_logo.png)

# Geog0027 Coursework: Part 1: ENVI Classification

## Data processing summary of requirements

* Download Landsat annual datasets for 1986 to present (or some suitable subset). **(of secondary importance, as you can copy the archive)**
* For one year (your choice), perform a supervised classification and an unsupervised classification using `envi` and assess the accuracy of the classifications; **(of primary importance)**
* Perform unsupervised classifications (clustering) of the time series of Landsat data, using an `envi` program that you will be provided with;
    * apply suitable class labels, and modify the number of classes as appropriate; **(of primary importance)**
* Calculate the area of urban land use for Shenzhen for each year **(of primary importance)**
* Calculate the area of urban land use change for Shenzhen for each year **(of primary importance)** 
* Estimate the area of agricultural land use for Shenzhen for each year **(of secondary importance)**



Prepared by:

[Dr Qingling Wu](http://www.geog.ucl.ac.uk/about-the-department/people/academic-staff/qingling-wu), [Prof Philip Lewis](http://www2.geog.ucl.ac.uk/~plewis/), [Dr Mathias Disney](http://www.geog.ucl.ac.uk/about-the-department/people/academic-staff/mat-disney)

Contact: [Professor Lewis](mailto:p.lewis@ucl.ac.uk)

-----------------


# Extraction of land use extent

Your task in this section is to calculate the area of land cover & land use (LULC) extent (in $m^2$) for each year of a series of Landsat imagery. If possible, you should provide an associated characterization of uncertainty in each of these areas. 

As a first step, we should *try* to quantify **at least three types** (urban, agricultural, and other) of land uses for each given year. A number of classification methods have been taught in this module, and Maximum Likelihood is a good starting point for multispectral TM imagery. There are also other image processing methods could help to identify land use classes (e.g. vegetation indices, filters, segmentation, etc.). Then, we can identify where changes have happened and how much land area has been transformed into urban built.  

For the modelling section of this report, it is **critical** that you extract a reasonable estimate of **urban** land cover for each year (it is not vital to do every year, but once you get started, this should not be too much bother). Agricultural land use should only be attempted if it proves feasible. If you do not believe it so, make a case in your report for not generating this cover class, and make sure you provide evidence to back this up. An acceptable excuse might,, for instance be that agriculture cannot be easily distinguished from other vegetation types in an annual dataset, but you would need to provide evidence of this. Also, some of the agricultural land use in the region is rice paddies, which might have rather similar signatures to other shallow water areas (if you want to claim that, provide evidence, e.g. spectra).

## Obtaining Landsat data

See the section [Google Download](DownloadEE.ipynb) for information on how to obtain and explore data. nYou should go through this section carefully, building your understanding of the datasets you are using. Take note of the 'hints' as to what might be interesting to explore (and put in your report). We suggest you access data using this approach. 

An alternative source of data is direct from the USGS, which you can explore in the [Download](Download.ipynb) page, including how to search only for the area we are interested in. It is not critical that you explore this for this practical, but you should find it of general use, and might, for example, use it to discuss issues (e.g. cloud) with using individual Landsat scenes, rather than the composites we make available via Google Earth Engine.

In any case, remember to write up each of the steps you go through in producing the input dataset for classification, and provide appropriate evidence. This will be in the `Methods` section, but you may decide to have a seperate `Remote Sensing Data Methods` section for clarity.

## Classifying the data: manual method

You should now have a set of (annual) image subsets (and associated masks, if required) of the area of interest.

You next need to generalise the datasets into classification maps. 

Although we will process the bulk of the data automatically, you will need to show that you can do a 'manual' classification. For this reason, we require you to **select one year (your choice) and perform a supervised classification and an unsupervised classification using `envi`**. 

You should **provide an assessment of the accuracy** of the classifications (truth tables), concentrating on the ability to distinuish **urban**, **agriculture** and **other** (you may go into more detail with other). 

You can use these accuracy assessments as an estimate of the uncertainty in the classifications when it comes to the modelling section (e.g. if you get an 80% accuracy on the urban class, you could suggest that there is roughly a 20% error on this class). 

You can either work on the whole extent of the downloaded dataset, or just use a mask for the Shenzhen region. Make sure you are clear about which of these you have chosen to do.

![](images/classy.png)

## Automated processing

We will use an ISOData clustering approach to process the full time series of data. You should make sure you are familiar with this approach and any issues in its use. As it is an unsupervised method, you will need to specify the number of classes you want. This choice should be based (at least initially) on your assessment of the datasets (e.g. the clusters you see in the NDVI / NDWI scatterplots):

![](images/2014scatter.png)

Note that the automated processing will use *only* the NDVI and NDWI channels of information ('bands' if you like), by default for ease of processing and interpretation. You do have control over which bands are used, so you should experiment with that (e.g. cluster using all bands, or just vegetation indices).

**Note that your results will be over-written each time you run the `envi` scripts** so you will want to make copies of results for a particular setting (e.g. bands. or number of clusters)



To use the automated processing, you should first download the [relevant files](http://www2.geog.ucl.ac.uk/~plewis/GEOG0027//classy.tar.Z) and place them in your data directory.


## Loading the automation software

Start `envi`.

You should have a window (terminal/shell) that shows the `ENVI>` prompt. This is where we will type 'envi` (actually, `IDL`) commands.

First, make sure you are in your data directory:


    ENVI> CD, '~/DATA/GEOG0027'
    
You can type Unix commands at this prompt if you put a `$` before the command, so if we want to check the files that are there, type:

    ENVI> $ls
    
You should see the following:
    
    1986		1994		2002		2010		2018
    1987		1995		2003		2011		2019
    1988		1996		2004		2012		classy.pro
    1989		1997		2005		2013		classy_lut1.dat
    1990		1998		2006		2014		
    1991		1999		2007		2015		
    1992		2000		2008		2016
    1993		2001		2009		2017
    
The automation code is in the file `classy.pro`, so we will get `envi` to load this:

    ENVI> .compile classy
    
This should respond with:

    % Compiled module: REAL_CLASSY.
    % Compiled module: CLASSY.
    % Compiled module: FIX_CLASS.
    % Compiled module: MAKE_GIF.
    % Compiled module: MAKE_MOVIE.
    
which is a list of the modules available to you. 

We can explore first `CLASSY`. 

    ; NAME:
    ;   classy
    ;
    ; PURPOSE:
    ;   Performs classification (clustering) on image
    ;   using ISOdata
    ;
    ; INPUTS:
    ;   input_file - name of envi image to read
    ;   nclasses - how many classesa (default 5)
    ;   bands    - band numbers to use (default [5,6])
    ;              use 0 for the 1st band. Max of 6
    ;
    ; OUTPUTS:
    ;    input_file + '_class'   - classification ENVI file
    ;
    ; AUTHOR:
    ;   P.Lewis, UCL 26 Jan 2019 (p.lewis@ucl.ac.uk)
    ;

Let's first look at a Landsat file `'2000/2000_Shenzhen`. We can load this into `envi` and visualise the Vegetation indices:

![](images/2000_Shenzhen.png)

We can see from this (and the scatter plot) that these two bands alone provide a good deal of discrimination of the main cover types. Let's have a go at generating 3 clusters:

For this module, you specify an `input_file` to process, and optionally the number of classes and the image bands to use (specify as e.g. [0,1,2,3,4,5,6] for all bands), so, for example:

    ENVI> CLASSY, '2000/2000_Shenzhen', 3, [5,6]
    
The result of running this should be an `envi` file `2000/2000_Shenzhen_class`. 

You can load that into `envi` using the usual menu system:

![](images/cluster3.png)

and we can see that this has been quite effective at pulling out 3 main cover types, namely water (`Class 1` in red), bare soil and urban (`Class 2` in green) and vegetation (`Class 3` in blue). You should refer back to the original RGB image to confirm these interpretations:

![](images/2000RGB.png)

You might decide that 3 clusters is sufficient: we have a vegetation class, even if its not explicitly agriculture, and we have an urban class (even though it probably includes areas of bare land as well).

If we decide that this is enough, we can process the whole time series on this basis.

First, we need to generate a text file to translate the class labels and colour to something more convenient.

We use a look up table (LUT) text file [classy_lut3.dat](files/classy_lut3.dat):

    Unclassified, 0,   0,   0
    Water, 0,   0,   255
    Urban, 200,   30, 0
    Vegetation, 20, 200, 0
    Masked Pixels, 64,  64,  64

This has, as well as the 3 clusters we wanted, a specification for `Unclassified` (class label 0) and `Masked Pixels` (class label 4) in case there are either of these.

Other than that, each line gives comma separated values of:

    NAME, R, G, B
    
where `NAME` is the class name we want to use, and `RGB` are numbers between 0 and 255 that specify the colour and intensity to use. For example `0, 255, 0` would be bright green, `255, 255, 0` would be bright yellow.

Lets use this now to re-code the dataset:

    ENVI> FIX_CLASS,'2000/2000_Shenzhen','classy_lut3.dat'
    
where we specify the file to operate on, and the LUT file.

This results in:

![](images/lut3.png)

which is an appropriate interpretation of the clusters.


**Note that if you change the wavebands used or the number of clusters, your image result will be over-written**. To avoid this, open the dataset in envi, and save with a different filename (eg one containing the bands used and number of classes)

Of course, we might decide to use more clusters (7 here) to get more subtle interpretations, but we will find the classes rather harder to interpret. Possibly the best answer lies somewhere in between. You will need to explore that and come to some (justified) conclusion.

![](images/cluster1.png)

Once we decide that we have probably got a good set up (and an appropriate LUT file), we can set the whole time series processing:

    ENVI> REAL_CLASSY,'.','classy_lut3.dat',[5,6]
    
This uses the module `REAL_CLASSY` and then specifies the directory to work in (`.` is the current directory, which is appropriate here), and then the LUT file to use, and the bands as above (use `[0,1,2,3,4,5,6]` for all bands). The number of classes to use is inferred from the number of entries in the LUT file.

This script will take a few minutes (or a little longer if more classes are specified), but will look over all of the directories containing the annual Landsat datasets (specified by year).

Once this is done, it applies the specified LUT to the classifications, and then generates a series of `gif` files that you can use in your report (each with the year labelled on the classification image).

Finally, an animated gif 'movie' is created:

![](images/class_movie.gif)



You should use this animation (and the associated gif files) to come to some conclusions about your selection of the number of classes. In this case, we can see that the `Water` class is useful and stable (although it includes rice paddies in the top left of the image). There seems to be some jumping around between the vegetation and urban classes however: we would generally expect an increase in the urban area and a decrease in the vegetation over time. Because we have only used 3 classes here, the clusters that form sometime include other features, such as bare land, and sometimes don't. This means that our class efinitions aren't very stable. 

It may be that some subset of the years that we have processed appears stable, and that may be enough to perform the modelling: in essence, we need *at least* 6 years of data over the time period for which we have the socioeconomic data to be able to provide an estimate of the 6 model parameters. Ideally, we should have at least twice that number. 

If not, you may wish to explore other numbers of clusters and the wavebands used. Make sure you note down your experimentation (with plenty of appropriate figures) in your report. Make sure you save each classified image set you derive if you change the bands or n umber of clusters.

![](images/class4.png)

## Pixel counting

Finally, you will need to provide a count of the total area of each class, with each pixel being 30 m x 30 m. To do this, you need to count how many pixels are in each class.

You can do this in `envi` following the menu items (right hand panel) `Classification -> Post Classification -> Class Statistics.

This will produce the data you need to track land cover for each date. Make a note of the pixel counts for each year.

![](images/stats.png)

Alternatively, the module `getstats` (in `envi`) will print the pixel counts for each class (assuming 8 or fewer classes). The format of the output is not great, but it contaions the information you need.

## Requirements

You *must* include an accuracy assessment for each **manual** classification you perform (so, for just one year). You must explain in the `Methods` section (possibly a seperate `Classification Methods` section) what classification algorithm(s) you used, and what settings you made (do not report defaults, just when you change things: this is to make your experiment **repeatable**). 

The `Methods` section or sub-section should contain 'full-sized' (on the page) pictures of the manual classification results, along with an appropriate table to interpret the colours. You must mention at this point if you have chosen to do any post-classification processing and demonstrate the steps taken. We mention this to allow you to more easily seperate which parts of the work should go into which section: all details and descriptions of the classifications, up to the point of producing a set of classified images and an appropriate dataset for validation must go into the `Methods` section (this is to do with allocation of marks, as much as anything else).

In the `Results` section, you must show each classified image in summary form e.g. in a table, for an overview, and provide the results of the classification accuracies. You should also report the number or proportion of pixels of each class, plotted as a function of year and present any other results you feel appropriate. Do *not* present many graphs that essentially show the same thing (unless indicated in these instructions), though you may include some in appendices (not simply to reduce the word count though).